In [35]:
# RandomForestClassifier(n_estimators=1000, criterion='gini', max_depth=None,  bootstrap=True, n_jobs=-1, random_state=0)
# SVC(C=1.0, kernel='linear', tol=0.001, cache_size=2000, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False)
# NearestNeighbors(n_neighbors=10, radius=1.0, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, n_jobs=None)
# MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)

In [36]:
import os
import re
import json
import exrex
import numpy
import Bio.SeqIO
import pandas as pd
from sklearn.svm import SVC
from joblib import Parallel, delayed
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [39]:
# Function to convert degenerative k-mers into a list of k-mers 
def convert_degenerative_k_mers(k_mer):
	k_mers = []
	temp_k_mer = ""
	
	for n in range(len(k_mer)): 
		if k_mer[n] == "A": temp_k_mer = temp_k_mer + "[A]"
		if k_mer[n] == "C": temp_k_mer = temp_k_mer + "[C]"
		if k_mer[n] == "G": temp_k_mer = temp_k_mer + "[G]"
		if k_mer[n] == "T": temp_k_mer = temp_k_mer + "[T]"
		if k_mer[n] == "N": temp_k_mer = temp_k_mer + "[ACGT]"
		if k_mer[n] == "X": temp_k_mer = temp_k_mer + "[ACGT]"
		if k_mer[n] == "V": temp_k_mer = temp_k_mer + "[^ACG]"
		if k_mer[n] == "H": temp_k_mer = temp_k_mer + "[^ACT]"
		if k_mer[n] == "D": temp_k_mer = temp_k_mer + "[^AGT]"
		if k_mer[n] == "B": temp_k_mer = temp_k_mer + "[^CGT]"
		if k_mer[n] == "M": temp_k_mer = temp_k_mer + "[AC]"
		if k_mer[n] == "W": temp_k_mer = temp_k_mer + "[AT]"
		if k_mer[n] == "R": temp_k_mer = temp_k_mer + "[AG]"
		if k_mer[n] == "K": temp_k_mer = temp_k_mer + "[GT]"
		if k_mer[n] == "S": temp_k_mer = temp_k_mer + "[CG]"
		if k_mer[n] == "Y": temp_k_mer = temp_k_mer + "[CT]"

	for i in exrex.generate(temp_k_mer): 
		k_mers.append(i)
	return k_mers

In [40]:
# Function to load test data from a fasta file
def loadDataTrain(file_path):
	# Initialize the data matrix
	D = []
	# Iterate through the fasta file
	for record in Bio.SeqIO.parse(file_path, "fasta"):
		# If there is a class label, save id, sequence and class label in the data list
		try: 
			indexes = [i for i, c in enumerate(record.description) if c == "|"]
			D.append([record.description, str(record.seq.upper()).replace('N',''), record.description[indexes[len(indexes)-1] +1 :]])
		# If there is a no class label, save id and sequence in the data list
		except: D.append([record.descrition, str(record.seq.upper())])
	# Return the data matrix
	return D

In [41]:
# Function to load test data from a fasta file
def loadDataTest(file_path, D_train):
	# Load train data to remove it from test data
	id = [d[0] for d in D_train]
	D_train.clear()
	# Initialize the data matrix
	D = []
	# Iterate through the fasta file
	for record in Bio.SeqIO.parse(file_path, "fasta"):
		# If there is a class label, save id, sequence and class label in the data list
		try: 
			# Get index of last separator
			indexes = [i for i, c in enumerate(record.description) if c == "|"]
			# Save id, sequence and class label
			if record.description not in id:
				D.append([record.description, str(record.seq.upper()), record.description[indexes[len(indexes)-1] +1 :]])
		# If there is a no class label, save id and sequence in the data list
		except: D.append([record.descrition, str(record.seq.upper())])
	# Return the data matrix
	return D

In [42]:
# Function to compute the occurence vector of sequence
def computeSequenceVector(d, K, k):
	# Generate an empty dictionary
	x = {}
	# Initialize the dictionary with targets as keys and 0 as value
	x = x.fromkeys(K.keys(), 0)
	# Go through the sequence 
	for i in range(0, len(d[1]) - k + 1, 1):
		# Try to increment the current k-mer value by 1
		try: x[d[1][i:i + k]] = x[d[1][i:i + k]] + 1
		# Pass if the k-mers does not exist
		except: pass
	# Return the vector and associated target
	return [list(x.values()), d[2]]

In [43]:
# Function to generate the samples matrix (X) and the target values (y)
def generateSamplesTargets(D, K, k):
	# Samples matrix
	X = []
	# Target values
	y = []
	# Iterate through the data
	data = Parallel(n_jobs = -1)(delayed(computeSequenceVector)(d, K, k) for d in D)
	# Add to the matrices
	for d in data: 
		X.append(d[0])
		y.append(d[1])

	# Convert to numpy array format
	X = numpy.asarray(X)
	y = numpy.asarray(y)
	# Return the samples matrix (X) and the target values (y)
	return X, y

In [44]:
# Compute and save performance metrics for STREME
data = {}

# Define the list of variants
variants = ["Alpha", "Beta", "Delta", "Epsilon", "Eta", "Gamma", "Iota", "Kappa", "Lambda", "Omicron"]
# Iterate throught prediction files
for n in range(1, 101): 
    y_true = []
    y_pred = []
    k_mers = []
    for variant in variants:
        with open("MEME_OOPS/" + str(n) + "/" + variant +  "/meme.txt") as file:
            for line in file: 
                if line.strip().count("MOTIF") and line.strip().count("width =   9"): 
                    line = line.strip()
                    start = 6
                    end = start + 9
                    k_mer =  line[start:end]
                    if bool(re.match('^[ACGT]+$', k_mer)) == True: 
                       k_mers.append(k_mer)
                    else: 
                        temp_k_mers = convert_degenerative_k_mers(k_mer)
                        for temp_k_mer in temp_k_mers: 
                            k_mers.append(temp_k_mer)
    k_mers = set(k_mers)

    K = dict.fromkeys(k_mers, 0)
    D_train = loadDataTrain("data/SARS-CoV-2_train_" + str(n) + ".fasta")
    X_train, y_train = generateSamplesTargets(D_train, K , 9)

    D_test = loadDataTest("data/SARS-CoV-2.fasta", D_train)
    X_test, y_test = generateSamplesTargets(D_test, K , 9)

    model = SVC(kernel = 'linear', C = 1, cache_size = 1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Compute the performance metrics
    f1 = f1_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    print(str(n) +") precision", precision, "recall", recall, "f1_score", f1)

    # Save the predictions
    file = open("MEME_OOPS/" + str(n) + "/prediction.csv", "w")
    file.write("id,y_pred\n")
    for i, y in enumerate(y_pred): file.write(D_test[i][0] + "," + y + "\n")
    file.close()


c:\Users\lebat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


1) precision 0.7898205778876003 recall 0.9019853461407499 f1_score 0.8204740848174261


c:\Users\lebat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


2) precision 0.828673606333948 recall 0.8558693387206349 f1_score 0.8385538633462819


c:\Users\lebat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


...

4) precision 0.7841080581574988 recall 0.8226172381758203 f1_score 0.7685715221216919


c:\Users\lebat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


5) precision 0.8547299574214515 recall 0.8794237308161076 f1_score 0.857900468153279


c:\Users\lebat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


6) precision 0.9033309499846643 recall 0.9011288907088998 f1_score 0.8737117829945221
7) precision 0.9298729862329278 recall 0.9951249940871699 f1_score 0.9554894886471793
8) precision 0.8254473243182296 recall 0.9911882327339956 f1_score 0.8620456732088778
9) precision 0.8778656912005556 recall 0.9913002233044818 f1_score 0.9078749907029957
10) precision 0.7611451454082478 recall 0.8583479369918813 f1_score 0.7448561908306359
11) precision 0.6504580332566288 recall 0.6496953704812215 f1_score 0.6275998574661082
12) precision 0.8978286554972381 recall 0.9308746977515243 f1_score 0.8715902732188894
13) precision 0.8585847767976205 recall 0.9625945603544004 f1_score 0.8596277598176257
14) precision 0.9750415149354271 recall 0.9961893453820647 f1_score 0.9846627116545692
15) precision 0.9763768681024849 recall 0.997240452240576 f1_score 0.9855078815529236
16) precision 0.9216416576166602 recall 0.9143924012008876 f1_score 0.9166027572538262
17) precision 0.8842265645973006 recall 0.975274